
# Задание:
----

<br>

Вам предоставлен анонимизированные результаты **реального A/B тестирования**, проходившего когда-то в нашем отделе (`ab_test_ensembles.csv`). В тесте было два варианта работы сервиса - `standard` и `svm_ensemble_v_1_22`, где работали несколько моделей классификации для целей сервиса.

<br>


------
Вам, как специалистам по машинному обучению предстоит ответить на **2 главных вопроса:**

1. Стоит ли нам оставить старый вариант работы сервиса или заменить его на вариант работы с моделями классификации (используем всю выборку 200к+ пользователей).
2. Кроме того, посчитайте вывод для типа пользователей (`user_type`). Стоит ли для новых (старых) пользователей оставить старый (новый) вариант работы сервиса.


<br>

Для того, чтобы освежить в памяти процесс тестирования статистических гипотез, непомню, что тестирование состоит из следующих частей:
1. Дизайн эксперимента.
2. Подготовка и запуск эксперимента.
3. Сбор данных и аналитика полученных данных.
4. Визуализация результатов тестирования.
5. Тестирование гипотез.
6. Вывод и интерпритация результатов.

-----

#### Критерии оценки задания:

1. **Первое, что будет проверяться - вывод полученных результатов**, в случае если выводы сделаны не правильно, задание считается проваленным и на этом этап проверки заканчивается (пропускаются этапы код-ревью, оформления и визуализации, качество кода).
2. В случае если результаты и интерпритация результатов оказались верны, проводится код-ревью и проверка этапа визуализаций, поиск проблемных точек, точек роста.
3. **Максимальный балл** который можно получить, выполнив текущее задание: **2 балла за 1 вопрос, 2 балла за 2 вопрос и 1 балл за эффективный и аккуратный код.


Удачи, примените все свои навыки, которые вам доступны на данный момент и покажите на что вы способны!

In [616]:
import pandas as pd
import numpy as np
import statistics
from scipy.stats import stats, norm, binom, beta, normaltest, probplot
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
np.random.seed(42)

#### 1. Знакомство с собранными данными по группам и базовая статистика.

In [617]:
# Dataset with AB testing results (2 groups -> standard and svm_ensemble_v_1_22)
data = pd.read_csv('./ab_test_ensembles.csv', index_col=False)
data.head()

,user_id,timestamp,group,variants,converted,location,age_group,user_type
0,9109b0dc-d393-497f-8d63-ba9a25dd16b4,2022-05-21 22:11:48.556739,control,standard,0,United Kingdom,18-25,registered_user
1,2430c3d2-f75b-4b31-8271-51b6a76c2652,2022-05-12 08:01:45.159739,control,standard,0,United Kingdom,42-49,registered_user
2,44788c4e-8dd2-4fad-b986-75e76f4adb64,2022-05-11 16:55:06.154213,treatment,svm_ensemble_v_1_22,0,United Kingdom,26-33,new_user
3,4699a417-506d-41b8-a354-6af6ad576963,2022-05-08 18:28:03.143765,treatment,svm_ensemble_v_1_22,0,United Kingdom,42-49,registered_user
4,304b0d28-bcdf-401a-9dff-66230d3ba0bc,2022-05-21 01:52:26.210827,control,standard,1,United Kingdom,42-49,registered_user


In [618]:
# check the features and data types of the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    294478 non-null  object
 1   timestamp  294478 non-null  object
 2   group      294478 non-null  object
 3   variants   294478 non-null  object
 4   converted  294478 non-null  int64 
 5   location   294478 non-null  object
 6   age_group  294478 non-null  object
 7   user_type  294478 non-null  object
dtypes: int64(1), object(7)
memory usage: 18.0+ MB


In [619]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
converted,294478.0,0.119659,0.324563,0.0,0.0,0.0,0.0,1.0


In [620]:
# Check if there is a corrupt or empty observations
data.isnull().sum()

user_id      0
timestamp    0
group        0
variants     0
converted    0
location     0
age_group    0
user_type    0
dtype: int64

In [621]:
# What kind of groups do we have in A/B testing
data.variants.unique()

array(['standard', 'svm_ensemble_v_1_22'], dtype=object)

In [622]:
# Distribution by groups
data.variants.value_counts()

variants
standard               147239
svm_ensemble_v_1_22    147239
Name: count, dtype: int64

In [623]:
# Distribution by groups in %
data.variants.value_counts(normalize=True)

variants
standard               0.5
svm_ensemble_v_1_22    0.5
Name: proportion, dtype: float64

In [624]:
# For simplicity let's rename the groups into A for control and B for treatment variants
data['variants'] = np.where(data['variants']== 'standard', 'A', 'B')

In [625]:
print(len(data.user_id.unique()),'rows of users data were collected.')

294478 rows of users data were collected.


In [626]:
if len(data[data['variants'] == 'A']) + len(data[data['variants'] == 'B']) == len(data.user_id.unique()):
  print("Each user is test with one version!")
  print(len(data[data['variants'] == 'A']),'users were in standard.')
  print(len(data[data['variants'] == 'B']),'users were in svm_ensemble_v_1_22.' )
else:
  print("There are user are test with more than one version")

Each user is test with one version!
147239 users were in standard.
147239 users were in svm_ensemble_v_1_22.


In [627]:
summary = data.groupby('variants').converted.agg(['sum'])
summary

,sum
variants,
A,17739
B,17498


In [628]:
summary = data.groupby(['user_type', 'variants']).agg({'converted': 'sum', 'user_id': 'count'}).rename(columns={'user_id' : 'users'})
summary

converted  users
user_type       variants                  
new_user        A              8935  73797
                B              8609  73441
registered_user A              8804  73442
                B              8889  73798

#### **Выводы:**

- Всего 294478 уникальных значений для каждого пользователя в данном эксперименте.

<br>

- Поровну 147239 пользователей распределенны в группы А и В.

<br>

- Суммарное количество converted составляет **17739** для группы А и **17498** для группы В.

<br>

- Среди новых пользователей **73797** принадлежит к группе А и **73441** к группе B.

<br>

- Среди старых пользователей **73442** принадлежить к группе A и **73798** к группе B.

<br>

- Среди новых пользователей суммарное количество converted в группе А составляет **8935** и **8609** для группы B.

<br>

- Среди старых пользователей суммарное количество converted в группе А составляет **8804** и **8889** для группы B.

#### 2. Обработка данных.

In [629]:
data.group.unique()

array(['control', 'treatment'], dtype=object)

Видим, что в датасете присутствует признак group со значениями control/treatment.<br>
Это разбиение датасета на наблюдения, где применяется ещё старый вариант работы сервиса (control), и на наблюдения, где применяется новый вариант работы сервиса (treatment).<br> 
Т.к. в нашем случае старый и новый варианты сервиса - это standard и svm_ensemble_v_1_22 соответственно, то наблюдениям группы control должен соответсвовать standard, а наблюдениям группы treatment должен соответствовать svm_ensemble_v_1_22.<br>
<br>
Проверим, нет ли данных, для которых это соответствие не выполняется.

In [630]:
count_invalid_variants = ((data.group=='control') & (data.variants!='A')).sum() + ((data.group=='treatment') & (data.variants!='B')).sum()

In [631]:
count_invalid_variants

3893

In [632]:
(count_invalid_variants / data.shape[0]) * 100

1.322000285250511

Видим, что есть наблюдения, в которых данное соответствие не выполняется.<br>
Т.к. процент таких наблюдений меньше 2%, то можем их убрать из выборки.

In [633]:
data = data.drop(data[((data.group=='control') & (data.variants!='A')) | ((data.group=='treatment') & (data.variants!='B'))].index)

In [634]:
((data.group=='control') & (data.variants!='A')).sum() + ((data.group=='treatment') & (data.variants!='B')).sum()

0

#### 3. Проведение анализа по метрике converted.

##### 3.1 Вынесем общий функционал

Т.к. мы ведём анализ дискретной величины (true/false), то будем применять непараметрический тесты **chi-squared test**, **Fisher’s exact test**, **Mann–Whitney U-test**.<br>
<br>
Т.к. нам необходимо будет провести анализ как для всей выборки, так и для каждого типа пользователей в отдельности, то для удобства вынесем функции для выполнения необходимых оценок в отдельные функции.

-------
**H0 (Null hypothesis)** : Не существует статистически значимой разницы между метриками в 2-х вариантах теста

P(converted rate of B) - P(converted rate of A) <= 0

**H1 (Alternative hypothesis)**:  Существует статистически значимая разница между 2-мя вариантами

P(converted rate of B) - P(converted rate of A) > 0


In [635]:
def get_converted_crosstab_per_variant(data):
    return pd.crosstab(data["variants"], data["converted"])

**chi-2 test**

- Уровень Значимости (Alpha) = 0.5 = вероятность отклонить нулевую гипотезу когда она истина (Type I)

In [636]:
# Chi-square test of independence of variables in a contingency table - Pearson's chi-square test or the chi-square test of association
from scipy.stats import chi2_contingency

def chi2_con_test(data):
    chi2, p, dof, ex = chi2_contingency(data)
    alpha = 0.05

    s = 'chi-squared test resulst: '
    s += '\np=%.4f , alpha=%.2f \n'%(p, alpha)
    if p > alpha:
        s += '\nFail to reject H0; Two version have no significant difference'
    else:
        s += '\nHave enough envidence to reject H0; Two version have a significant difference'

    return s

In [637]:
def get_chi2_con_test_results(data):
    return chi2_con_test(get_converted_crosstab_per_variant(data))

**Mann–Whitney U-test**

- Уровень Значимости (Alpha) = 0.5 = вероятность отклонить нулевую гипотезу когда она истина (Type I)

In [638]:
from scipy.stats import mannwhitneyu

def mannwhitney_test(value1, value2):

    stat, p=mannwhitneyu(value1, value2)
    alpha=0.05

    s = "Mann–Whitney U-test test results:"
    s += '\np=%.4f , alpha=%.2f \n'%(p, alpha)
    if p > alpha:
        s += '\nFail to reject H0; Two version have no significant difference'
    else:
        s += '\nHave enough envidence to reject H0; Two version have a significant difference'

    return s

**Fisher’s exact test**

- Уровень Значимости (Alpha) = 0.5 = вероятность отклонить нулевую гипотезу когда она истина (Type I)

In [639]:
from scipy.stats import fisher_exact

def fisher_exact_test(data):

    # Perform Fisher's exact test
    odds_ratio, p = fisher_exact(data)
    alpha=0.05

    s = "Fisher’s exact test results:"
    s += '\np=%.4f , alpha=%.2f \n'%(p,alpha)
    if p > alpha:
        s += '\nFail to reject H0; Two version have no significant difference'
    else:
        s += '\nHave enough envidence to reject H0; Two version have a significant difference'

    return s

In [640]:
def get_fisher_exact_test_results(data):
    return fisher_exact_test(get_converted_crosstab_per_variant(data))

**Function to get all tests results**

In [641]:
def calc_converted_test_results(data):
    # what is the overall converted rate?
    overall_converted_rate = data.converted.sum()/data.converted.count()*100

    # what is converted rate for each version?
    converted_rate_percents_per_variant = data.groupby('variants').converted.sum()/data.groupby('variants').converted.count()*100

    converted_rate_per_variant = data.groupby('variants').agg({'converted':'mean'})

    # chi2 test results
    chi2_test_results = get_chi2_con_test_results(data)

    mannwhitneyu_test_results = mannwhitney_test(data[data.variants=='A'].converted, data[data.variants=='B'].converted)

    fisher_test_results = get_fisher_exact_test_results(data)
    
    return (overall_converted_rate, converted_rate_percents_per_variant, converted_rate_per_variant, chi2_test_results, mannwhitneyu_test_results, fisher_test_results)

##### 3.2 Проведение анализа метрики converted по всей выборке.

In [642]:
( 
    overall_converted_rate, 
    converted_rate_percents_per_variant, 
    converted_rate_per_variant, 
    chi2_test_results,
    mannwhitneyu_test_results,
    fisher_test_results
) = calc_converted_test_results(data)

In [643]:
overall_converted_rate

11.959667567149026

In [644]:
converted_rate_percents_per_variant

variants
A    12.038630
B    11.880725
Name: converted, dtype: float64

In [645]:
converted_rate_per_variant

,converted
variants,
A,0.120386
B,0.118807


In [646]:
print(chi2_test_results)

chi-squared test resulst: 
p=0.1916 , alpha=0.05 

Fail to reject H0; Two version have no significant difference


In [647]:
print(mannwhitneyu_test_results)

Mann–Whitney U-test test results:
p=0.1897 , alpha=0.05 

Fail to reject H0; Two version have no significant difference


In [648]:
print(fisher_test_results)

Fisher’s exact test results:
p=0.1905 , alpha=0.05 

Fail to reject H0; Two version have no significant difference


**Выводы:**
- Общий показатель метрики converted по всем данным **11.966%**; группа A **12.039%** и группа B **11.881%**
- У группы А показатели метрики converted выше чем у группы В.
- У обоих вариантов нет статистически значимых изменений в метрике converted.



##### 3.3 Проведение анализа метрики converted для старых пользователей.

In [649]:
( 
    overall_converted_rate, 
    converted_rate_percents_per_variant, 
    converted_rate_per_variant, 
    chi2_test_results,
    mannwhitneyu_test_results,
    fisher_test_results
) = calc_converted_test_results(data[data.user_type=='registered_user'])

In [650]:
overall_converted_rate

12.007955296190266

In [651]:
converted_rate_percents_per_variant

variants
A    11.987638
B    12.028176
Name: converted, dtype: float64

In [652]:
converted_rate_per_variant

,converted
variants,
A,0.119876
B,0.120282


In [653]:
print(chi2_test_results)

chi-squared test resulst: 
p=0.8184 , alpha=0.05 

Fail to reject H0; Two version have no significant difference


In [654]:
print(mannwhitneyu_test_results)

Mann–Whitney U-test test results:
p=0.8121 , alpha=0.05 

Fail to reject H0; Two version have no significant difference


In [655]:
print(fisher_test_results)

Fisher’s exact test results:
p=0.8150 , alpha=0.05 

Fail to reject H0; Two version have no significant difference


**Вывод:**
- Общий показатель метрики converted для старых пользователей **12.008%**; группа A **11.988%** и группа B **12.029%**
- У группы B показатели метрики converted выше чем у группы A.
- У обоих вариантов нет статистически значимых изменений в метрике converted для старых пользователей.



##### 3.4 Проведение анализа метрики converted для новых пользователей.

In [656]:
( 
    overall_converted_rate, 
    converted_rate_percents_per_variant, 
    converted_rate_per_variant, 
    chi2_test_results,
    mannwhitneyu_test_results,
    fisher_test_results
) = calc_converted_test_results(data[data.user_type=='new_user'])

In [657]:
overall_converted_rate

11.911366874780585

In [658]:
converted_rate_percents_per_variant

variants
A    12.089407
B    11.732568
Name: converted, dtype: float64

In [659]:
converted_rate_per_variant

,converted
variants,
A,0.120894
B,0.117326


In [660]:
print(chi2_test_results)

chi-squared test resulst: 
p=0.0365 , alpha=0.05 

Have enough envidence to reject H0; Two version have a significant difference


In [661]:
print(mannwhitneyu_test_results)

Mann–Whitney U-test test results:
p=0.0358 , alpha=0.05 

Have enough envidence to reject H0; Two version have a significant difference


In [662]:
print(fisher_test_results)

Fisher’s exact test results:
p=0.0359 , alpha=0.05 

Have enough envidence to reject H0; Two version have a significant difference


**Вывод:**
- Общий показатель метрики converted по всем данным **11.911%**; группа A **12.089%** и группа B **11.733%**
- У группы А показатели метрики converted выше чем у группы В.
- Для новых пользователей есть статистически значимые изменения в метрике converted между группами A и B.



#### 4. Подведение итогового вывода.

Т.к. для новых пользователей наблюдается значительные изменения в метрике converted между группами A и B при том, что со старым вариантом работы сервиса показатели метрики converted выше, и при этом для всей выборки и отдельно для старых пользователей статистически значимых изменений в метрике converted между группами A и B не наблюдается, то делаем выввод, что **целесообразней будет оставить старый вариант работы сервиса**.